In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # train_data processing, CSV file I/O (e.g. pd.read_csv)

# Input train_data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/dataset/HousePricePrediction.xlsx
/kaggle/input/dataset-information-steps/steps.txt
/kaggle/input/dataset-information-steps/dataset explaination.md


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
data = pd.read_excel(r"/kaggle/input/train_dataset/HousePricePrediction.xlsx")

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/train_dataset/HousePricePrediction.xlsx'

In [ ]:
data.head()

In [ ]:
data.shape

In [ ]:
data.info()

In [ ]:
def train_datatypes(data):
    obj = data.dtypes == "object"
    ints = data.dtypes == "int64"
    floats = data.dtypes == "float64"

    print("Object columns:", data.columns[obj].tolist(),':',len(data.columns[obj].tolist()),"\n")
    print("Integer columns:", data.columns[ints].tolist(),':',len(data.columns[ints].tolist()),"\n")
    print("Float columns:", data.columns[floats].tolist(),':',len(data.columns[floats].tolist()),"\n")

train_datatypes(train_data)

In [ ]:
data.describe()


In [ ]:
data.isnull().sum()

In [ ]:
data.head()

In [ ]:
def get_uniques(df, columns):
   
    uniques = {}
    for col in columns:
        if col in df.columns:
            uniques[col] = df[col].dropna().unique().tolist()
        else:
            uniques[col] = None  # Or []/warning depending on your preference
    return uniques

col = ['MSSubClass','MSZoning','LotConfig','BldgType',
       'OverallCond','Exterior1st']
#'LotArea','YearBuilt','YearRemodAdd', 'BsmtFinSF2','TotalBsmtSF'

uniques = get_uniques(data, col)
for k, v in uniques.items():
    print(f"{k}: {v}\n")

In [ ]:
train_data = data.iloc[0:1460,:]
test_data = data.iloc[1460:,:12]

In [ ]:
train_data

In [ ]:
test_data

In [ ]:
num_cols = ['MSSubClass','LotArea','OverallCond','YearBuilt',
            'YearRemodAdd','BsmtFinSF2','TotalBsmtSF']


# Heatmap of correlations
plt.figure(figsize=(7,5))
sns.heatmap(train_data[num_cols + ['SalePrice']].corr(), annot=True, cmap='coolwarm')
plt.title('Correlation Matrix')
plt.show()


In [ ]:
# Scatter plots for each numeric column vs SalePrice
for col in num_cols:
    plt.figure(figsize=(5,4))
    plt.scatter(train_data[col], train_data['SalePrice'], alpha=0.5)
    plt.xlabel(col)
    plt.ylabel('SalePrice')
    plt.title(f'{col} vs SalePrice')
    plt.tight_layout()
    plt.show()

In [ ]:
cat_cols = ['MSSubClass', 'OverallCond']

for col in cat_cols:
    # 3.1 Value counts
    print(f"\n-- {col} value counts:")
    print(train_data[col].value_counts())

    # 3.2 Boxplot of SalePrice by category
    plt.figure(figsize=(6,4))
    sns.boxplot(x=col, y='SalePrice', train_data=train_data)
    plt.title(f'SalePrice by {col}')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

In [ ]:
train_data.head()

In [ ]:
train_data.isnull().sum()


In [ ]:
train_data['Exterior1st']=train_data['Exterior1st'].fillna(train_data['Exterior1st'].mode()[0])
train_data['MSZoning']=train_data['MSZoning'].fillna(train_data['MSZoning'].mode()[0])


In [ ]:

train_data['BsmtFinSF2']=train_data['BsmtFinSF2'].fillna(train_data['BsmtFinSF2'].mean())
train_data['TotalBsmtSF']=train_data['TotalBsmtSF'].fillna(train_data['TotalBsmtSF'].mean())

In [ ]:
train_data.isnull().sum()


In [ ]:
train_data= train_data.drop(columns=['Id'])


In [ ]:

#Object columns: ['MSZoning', 'LotConfig', 'BldgType', 'Exterior1st']
encoding =['MSZoning', 'LotConfig', 'BldgType', 'Exterior1st']
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

for col in encoding:
    train_data[col] = le.fit_transform(train_data[col])

In [ ]:
train_data

In [ ]:
train_data=train_data.drop(['Id'],axis=1)


In [ ]:
X= train_data.drop(columns=['SalePrice'])
Y= train_data['SalePrice']

In [ ]:
Y

In [ ]:
X

In [ ]:
from sklearn.model_selection import train_test_split
X_TRAIN, X_TEST, Y_TRAIN,Y_TEST = train_test_split(
    X,Y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.linear_model import LinearRegression

ln = LinearRegression()
ln.fit(X_TRAIN, Y_TRAIN)

y_pred_LR = ln.predict(X_TEST)

In [ ]:
from sklearn.metrics import r2_score

r2_scoreln = r2_score(Y_TEST,y_pred)
print(r2_scoreln)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

model_RFR = RandomForestRegressor(n_estimators=10)
model_RFR.fit(X_TRAIN, Y_TRAIN)
Y_pred_RF = model_RFR.predict(X_TEST)

print(mean_absolute_percentage_error(Y_TEST,Y_pred_RF))
r2_scoreRF = r2_score(Y_TEST,Y_pred_RF)
print(r2_scoreRF)

In [ ]:
from sklearn import svm
from sklearn.svm import SVC
from sklearn.metrics import mean_absolute_percentage_error

model_SVR = svm.SVR()
model_SVR.fit(X_TRAIN, Y_TRAIN)
Y_pred_SVM = model_SVR.predict(X_TEST)

print(mean_absolute_percentage_error(Y_TEST, Y_pred_SVM))
print(r2_score(Y_TEST,Y_pred_SVM))


In [ ]:
#PREDICTION

predict_house_price = pd.DataFrame([['60','3','8450','4','0','5','2003','2003','12','0.0','856.0']],
                                  columns=['MSSubClass', 'MSZoning', 'LotArea', 'LotConfig', 'BldgType',
       'OverallCond', 'YearBuilt', 'YearRemodAdd', 'Exterior1st', 'BsmtFinSF2',
       'TotalBsmtSF'])

In [ ]:
''''from sklearn.preprocessing import StandardScaler
sc2 = StandardScaler()
pred_new_testing =sc.fit_transform(predict_house_price)
''''


In [ ]:
model_RFR.predict(pred_new_testing)

In [ ]:


X_train=train_data.drop(['SalePrice'],axis=1)

Y_train=train_data['SalePrice']

In [ ]:
Y_train

In [ ]:
Y_train = np.array(train_data['SalePrice']).reshape(-1,1)


In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
Y_train = sc.fit_transform(Y_train)

In [ ]:
Y_train 


In [ ]:
test_data

In [ ]:

#Object columns: ['MSZoning', 'LotConfig', 'BldgType', 'Exterior1st']
encoding =['MSZoning', 'LotConfig', 'BldgType', 'Exterior1st']
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

for col in encoding:
    test_data[col] = le.fit_transform(test_data[col])

In [ ]:
test_data.isnull().sum()

In [ ]:
test_data['BsmtFinSF2']=test_data['BsmtFinSF2'].fillna(test_data['BsmtFinSF2'].mean())
test_data['TotalBsmtSF']=test_data['TotalBsmtSF'].fillna(test_data['TotalBsmtSF'].mean())

In [ ]:
test_data.isnull().sum()

In [ ]:
X_test =test_data.copy()

In [ ]:
X_test=test_data.drop(['Id'],axis=1)


In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_test = sc.fit_transform(X_test)

In [ ]:
X_test

In [ ]:
from sklearn.linear_model import LinearRegression

ln = LinearRegression()
ln.fit(X_train, Y_train)

y_pred = ln.predict(X_test)

In [ ]:
y_pred

In [ ]:
y_pred.shape

In [ ]:
y_pred_original = sc.inverse_transform(y_pred)


In [ ]:
new_BsmtFinSF2=train_data['BsmtFinSF2']

In [ ]:
sns.displot(train_data['BsmtFinSF2'])

In [ ]:
#to see outlier
sns.boxplot(train_data['BsmtFinSF2'])

In [ ]:
outlier=[]

def detect_outlier(train_dataset):

    threshold=5
    mean=np.mean(train_dataset)
    std=np.std(train_dataset)


    for i in train_dataset:
        z_score = (i-mean)/std

        if np.abs(z_score)> threshold:
            outlier.append(i)
    return outlier



In [ ]:
outlier_output=detect_outlier(new_BsmtFinSF2)

In [ ]:
outlier_output

In [ ]:
import pandas as pd

def detect_outliers_iqr(train_data, column):
    Q1 = train_data[column].quantile(0.25)
    Q3 = train_data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # Select rows where the column value is an outlier
    outliers = train_data[(train_data[column] < lower_bound) | (train_data[column] > upper_bound)]
    
    print(f"Number of outliers in '{column}': {outliers.shape[0]}")
    print(f"Outlier values in '{column}':\n", outliers[[column]])
    
    return outliers

# Example usage:
train_data = pd.read_excel('/kaggle/input/train_dataset/HousePricePrediction.xlsx')

# Detect and display outliers in SalePrice
outliers_saleprice = detect_outliers_iqr(train_data, 'SalePrice')

# Detect and display outliers in LotArea
outliers_lotarea = detect_outliers_iqr(train_data, 'LotArea')

# Detect and display outliers in TotalBsmtSF
outliers_bsmt = detect_outliers_iqr(train_data, 'TotalBsmtSF')
